In [1]:
import torch 
import torch.nn as nn 

In [ ]:
'''
Attention mechanism the text generating decoder part of the network can acesss all input token selectively this mean that some input tokens are more important than others for generating 
a given output token 
This means that some input toekns are more importaant than others for genrating a given output token .
'''